<a href="https://colab.research.google.com/github/DenysNunes/data-examples/blob/main/spark/basic/dataframes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Init spark**

In [1]:
!pip install -q pyspark==3.1.1

from pyspark.sql import SparkSession


spark = SparkSession \
    .builder \
    .master('local[*]') \
    .appName("New Session Example") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .enableHiveSupport() \
    .getOrCreate()

     |████████████████████████████████| 212.3 MB 15 kB/s 
     |████████████████████████████████| 198 kB 20.3 MB/s 


## **Creating a singe DF**

In [2]:
from pyspark.sql.types import Row

rows = [
        Row(id=1, name='Jonh'),
        Row(id=2, name='Maria'),
        Row(id=3, name='Ben')
]

df = spark.createDataFrame(rows)

df.show()

+---+-----+
| id| name|
+---+-----+
|  1| Jonh|
|  2|Maria|
|  3|  Ben|
+---+-----+

